# __ETL__ _(Extract, Transform, Load)_

## Introducción

Este notebook se enfoca en el proceso de **ETL** utilizando datos extraídos de las plataformas Yelp y Google Maps. Este proceso implica una _extracccion,transformación y carga_ de los datos con el objetivo de prepararlos para análisis posteriores. Este paso es crucial en cualquier proyecto de ciencia de datos para garantizar la calidad y utilidad de los datos.

## Configuraciones Globales e Importaciones

En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [1]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [2]:
import os # Proporciona funciones para interactuar con el sistema operativo.
import requests # Se utiliza para realizar solicitudes HTTP.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import pyspark.pandas as ps # Proporciona una interfaz para trabajar con datos en Spark utilizando el formato de DataFrame de pandas.
import json # Se utiliza para trabajar con datos en formato JSON.
from pyspark.sql import SparkSession # Se utiliza para crear una instancia de SparkSession, que es la entrada principal para trabajar con Spark SQL.
from pyspark.sql import functions as F #  Proporciona funciones para trabajar con datos en Spark DataFrame.
from pyspark.sql.functions import array_contains # Esta función se utiliza para filtrar los datos basados en la presencia de un valor en un array.
from pyspark.sql.functions import sum, col # Se utiliza para acceder a una columna en un DataFrame de Spark.
from pyspark.sql.functions import split, substring, concat_ws
from pyspark.sql.functions import expr

In [4]:
# Crear una sesión de Spark
spark = SparkSession.builder\
        .master("local")\
        .appName("ETL-review")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# YELP

**Dataset:** REVIEW.JSON

DECLARACIÓN DE LA RUTA DE LOS DATA SET

In [5]:
#Se lee el archivo review de yelp

df = spark.read.json("C:/Users/Usuario/Desktop/Proyecto Final/Yelp/review.json")

In [6]:
#Muestra eñ dataframe
df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [7]:
# Convierte la columna 'date' al formato datetime 
df = df.withColumn('date', df['date'].cast('timestamp'))

In [8]:
# Ordena el DataFrame por 'date' 
df = df.orderBy('date')

In [9]:
# Cambia el nombre de la columna 'stars' por 'rating' 
df = df.withColumnRenamed('stars', 'rating')

In [10]:
# Cambia el nombre de la columna 'text' por 'review' 
df = df.withColumnRenamed('text', 'review')

In [11]:
# Reordena las columnas 
df = df.select('review_id', 'date', 'user_id', 'business_id', 'review', 'rating', 'cool', 'useful', 'funny')

In [12]:
df.show()

+--------------------+-------------------+--------------------+--------------------+--------------------+------+----+------+-----+
|           review_id|               date|             user_id|         business_id|              review|rating|cool|useful|funny|
+--------------------+-------------------+--------------------+--------------------+--------------------+------+----+------+-----+
|IykJMMZgbNcUndwf1...|2005-02-16 03:23:22|3zBJUlWtPNoZ0uN83...|2bXm0SynOfxDzfrdr...|It's not chicago ...|   4.0|   0|     0|    0|
|-O5ng1XLox6uEr4uI...|2005-02-16 03:29:39|3zBJUlWtPNoZ0uN83...|3g6XqkBikTTbZmTuk...|Great service.  T...|   4.0|   0|     1|    0|
|g80vzN72iU03Wh0fS...|2005-02-16 04:06:26|3zBJUlWtPNoZ0uN83...|PP3BBaVxZLcJU54uP...|These guys really...|   5.0|   0|     0|    0|
|7Dcrt0Oz0hikA8obG...|2005-03-01 16:57:17|XCsZ3hWa_6oP1WkWv...|U3grYFIeu6RgAAQgd...|Words cannot desc...|   5.0|   0|     0|    0|
|WC9q5vhQlQkLK05kE...|2005-03-01 16:59:37|XCsZ3hWa_6oP1WkWv...|Aes-0Q_guDeYewMap...

In [13]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- review: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- cool: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)



## Carga de nuestro archivo

In [ ]:
ruta_nuevo_json = "C:/Users/Usuario/Desktop/Proyecto Final/PF_Google_yelp_Map/Notebook/review_limpio.json"
df.coalesce(1).write.json(ruta_nuevo_json)